In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime 
import seaborn as sns
%matplotlib inline

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.utils import shuffle
import statsmodels.api as sm

/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
def set_data(file):
    bikeshare_machine = pd.read_csv(file, 
                        parse_dates=['Start date', 'End date', 'start_date_short', 'end_date_short'])
    bikeshare_machine.drop('Unnamed: 0', 1, inplace=True)
    included_cols = ['start_station','end_station','Member Type','time_diff','season','mnth','holiday',
                     'weekday','workingday','weathersit','temp','hum','windspeed','miles',
                     'rush_hour','metro_dist']
    bikeshare_machine = bikeshare_machine[included_cols]
    bikeshare_machine['season'] = bikeshare_machine['season'].astype('category')
    bikeshare_machine['mnth'] = bikeshare_machine['mnth'].astype('category')
    bikeshare_machine['holiday'] = bikeshare_machine['holiday'].astype('category')
    bikeshare_machine['weekday'] = bikeshare_machine['weekday'].astype('category')
    bikeshare_machine['workingday'] = bikeshare_machine['workingday'].astype('category')
    bikeshare_machine['weathersit'] = bikeshare_machine['weathersit'].astype('category')
    bikeshare_machine['Member Type'] = bikeshare_machine['Member Type'].astype('category')
    bikeshare_machine['start_station'] = bikeshare_machine['start_station'].astype('category')
    bikeshare_machine['end_station'] = bikeshare_machine['end_station'].astype('category')
    bikeshare_machine['rush_hour'] = bikeshare_machine['rush_hour'].astype('category')
    col_names = ['start_station', 'end_station','member_type','time_diff','season','month','holiday',
             'weekday','work_day','weather_cat','temperature','humidity','windspeed','miles','rush_hour',
                'metro_dist']
    bikeshare_machine.columns = col_names
    tmin = -8
    tmax = 39
    hum_max = 100
    wind_max = 67
    bikeshare_machine['temp'] = bikeshare_machine['temperature'] * (tmax - tmin) + tmin
    bikeshare_machine['hum'] = bikeshare_machine['humidity'] * 100
    bikeshare_machine['wind'] = bikeshare_machine['windspeed'] * 67
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['rush_hour','member_type','holiday','work_day'], drop_first=True)
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['start_station','end_station','season','month','weekday','weather_cat'])
    bikeshare_machine = shuffle(bikeshare_machine)
    return bikeshare_machine

In [4]:
file = '/Users/matthewcassi/Documents/Bike-Sharing-Dataset/Bikeshare_Time_Prediction/reg_metro_rush.csv'
bikeshare_machine = set_data(file)

In [5]:
bikeshare_machine = bikeshare_machine.drop(['temperature', 'humidity', 'windspeed'], 1)
bikeshare_machine = bikeshare_machine.rename(columns = {'member_type_Registered':'member_type'})
bikeshare_machine.head()

,time_diff,miles,metro_dist,temp,hum,wind,rush_hour_1,holiday_1,work_day_1,start_station_10th & Monroe St NE,...,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weather_cat_1,weather_cat_2,weather_cat_3
126678,21.500,1.185929,0.023451,26.466651,70.7500,11.541554,0,0,1,0,...,0,0,0,1,0,0,0,1,0,0
830121,4.300,0.501247,0.023451,6.844151,54.3333,14.125543,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
647921,9.833,1.313535,0.250981,7.470849,50.2083,15.041232,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
896327,7.017,1.115181,0.250981,13.345849,55.4583,12.791171,0,0,1,0,...,0,0,0,0,1,0,0,1,0,0
752470,5.367,0.524107,0.252381,16.009151,68.4167,1.500244,0,0,1,0,...,0,0,0,0,0,1,0,1,0,0


### Model 1 - Remove Some Variables that are correlated

In [6]:
# Leave workday, drop weekdays, leave season, drop month
# Workday overlaps with workday/not workday and months overlap with seasons
remove_cols = ['weekday_0', 'weekday_1','weekday_2','weekday_3','weekday_4','weekday_5','weekday_6', 
              'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9',
              'month_10','month_11','month_12','time_diff']
X1 = np.matrix(bikeshare_machine.drop(remove_cols, 1))
y1 = bikeshare_machine['time_diff']

In [7]:
# Split the data into training and testing sets and check the shape
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = 0.25)
X1_train.shape, X1_test.shape, y1_train.shape, y1_test.shape

((762165, 277), (254056, 277), (762165,), (254056,))

In [8]:
# Fit model
model1 = LinearRegression()
model1.fit(X1_train, y1_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [9]:
pred1 = model1.predict(X1_test)
cvscores_model1 = cross_val_score(model1, X1_train, y1_train, cv=5)
model1_r = model1.score(X1_test, y1_test)
model1_mse = mean_squared_error(y1_test, pred1)
model1_rmse = np.sqrt(model1_mse)
adjustedr1 = 1 - (1-model1_r)*(len(y1_test)-1)/(len(y1_test)-X1_test.shape[1]-1)

In [10]:
cvscores_model1, model1_r, adjustedr1, model1_mse, model1_rmse

(array([  4.52470992e-01,   4.54454557e-01,   4.57145487e-01,
         -5.24227558e+07,   4.60146362e-01]),
 0.4553257656057792,
 0.45473125086089505,
 35.00140200917555,
 5.9161982733150138)

In [11]:
model1_sm = sm.OLS(y1_train, X1_train.astype(float)).fit()
model1_sm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              time_diff   R-squared:                       0.459
Model:                            OLS   Adj. R-squared:                  0.458
Method:                 Least Squares   F-statistic:                     2373.
Date:                Thu, 14 Dec 2017   Prob (F-statistic):               0.00
Time:                        19:13:04   Log-Likelihood:            -2.4353e+06
No. Observations:              762165   AIC:                         4.871e+06
Df Residuals:                  761892   BIC:                         4.874e+06
Df Model:                         272                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             7.1218      0.010    706.048      0.000       7.102       7.142
x2             0.0792      0.136      0.582      0.561      -0.188       0.346
x3             0.0549      0.001     38.758      0.000       0.052       0.058
x4            -0.0013      0.001     -1.898      0.058      -0.003    4.11e-05
x5            -0.0111      0.001     -7.648      0.000      -0.014      -0.008
x6            -1.2969      0.020    -63.524      0.000      -1.337      -1.257
x7            -0.3942      0.049     -7.988      0.000      -0.491      -0.298
x8            -0.8304      0.017    -48.494      0.000      -0.864      -0.797
x9            -1.5870      0.155    -10.218      0.000      -1.891      -1.283
x10           -0.6645      0.090     -7.380      0.000      -0.841      -0.488
x11            2.1959      0.098     22.467      0.000       2.004       2.388
x12           -0.2904      0.176     -1.654      0.098      -0.635       0.054
x13           -1.4448      0.082    -17.558      0.000      -1.606      -1.284
x14            0.8004      0.229      3.501      0.000       0.352       1.249
x15           -0.8228      0.228     -3.604      0.000      -1.270      -0.375
x16           -0.7765      0.098     -7.955      0.000      -0.968      -0.585
x17           -0.2468      0.113     -2.185      0.029      -0.468      -0.025
x18            0.1673      0.089      1.877      0.060      -0.007       0.342
x19            1.1642      0.103     11.340      0.000       0.963       1.365
x20           -0.1140      0.097     -1.175      0.240      -0.304       0.076
x21            0.5143      0.093      5.528      0.000       0.332       0.697
x22           -2.0524      0.083    -24.782      0.000      -2.215      -1.890
x23           -0.7137      0.079     -9.042      0.000      -0.868      -0.559
x24           -0.3937      0.076     -5.161      0.000      -0.543      -0.244
x25           -0.7683      0.076    -10.061      0.000      -0.918      -0.619
x26           -2.1586      0.097    -22.225      0.000      -2.349      -1.968
x27           -2.1865      0.170    -12.876      0.000      -2.519      -1.854
x28            1.1872      0.180      6.581      0.000       0.834       1.541
x29            0.1216      0.071      1.706      0.088      -0.018       0.261
x30           -1.3701      0.191     -7.160      0.000      -1.745      -0.995
x31           -1.8176      0.077    -23.728      0.000      -1.968      -1.667
x32           -0.0036      0.071     -0.050      0.960      -0.142       0.135
x33           -0.0693      0.098     -0.710      0.478      -0.261       0.122
x34           -0.1037      0.084     -1.229      0.219      -0.269       0.062
x35           -0.0089      0.148     -0.060      0.952      -0.300       0.282
x36           -0.1815      0.107     -1.689      0.091      -0.392       0.029
x3

### Model 2 - Try reverse of Model 1

In [12]:
# Drop workday, leave weekdays, drop season, leave month
# Workday overlaps with workday/not workday and months overlap with seasons
remove_cols = ['work_day_1','season_1', 'season_2', 'season_3', 'season_4','time_diff']
X2 = np.matrix(bikeshare_machine.drop(remove_cols, 1))
y2 = bikeshare_machine['time_diff']

In [13]:
# Split the data into training and testing sets and check the shape
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size = 0.25)
X2_train.shape, X2_test.shape, y2_train.shape, y2_test.shape

((762165, 291), (254056, 291), (762165,), (254056,))

In [14]:
# Fit model
model2 = LinearRegression()
model2.fit(X2_train, y2_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [15]:
pred2 = model2.predict(X2_test)
cvscores_model2 = cross_val_score(model2, X2_train, y2_train, cv=5)
model2_r = model2.score(X2_test, y2_test)
model2_mse = mean_squared_error(y2_test, pred2)
model2_rmse = np.sqrt(model2_mse)
adjustedr2 = 1 - (1-model2_r)*(len(y2_test)-1)/(len(y2_test)-X2_test.shape[1]-1)

In [16]:
cvscores_model2, model2_r, adjustedr2, model2_mse, model2_rmse

(array([  4.60029070e-01,  -1.31128127e+11,  -5.76712019e+09,
          4.54460599e-01,   4.63087111e-01]),
 0.45719244619304167,
 0.45656998990232345,
 34.709163208133781,
 5.8914483115897553)

In [17]:
# Use statsmodel to check the statistical side of the model
model2_sm = sm.OLS(y2_train, X2_train.astype(float)).fit()
model2_sm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              time_diff   R-squared:                       0.459
Model:                            OLS   Adj. R-squared:                  0.459
Method:                 Least Squares   F-statistic:                     2268.
Date:                Thu, 14 Dec 2017   Prob (F-statistic):               0.00
Time:                        19:15:33   Log-Likelihood:            -2.4357e+06
No. Observations:              762165   AIC:                         4.872e+06
Df Residuals:                  761879   BIC:                         4.875e+06
Df Model:                         285                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             7.1207      0.010    705.373      0.000       7.101       7.140
x2             0.0861      0.137      0.628      0.530      -0.183       0.355
x3             0.0443      0.002     21.702      0.000       0.040       0.048
x4            -0.0035      0.001     -4.870      0.000      -0.005      -0.002
x5            -0.0146      0.001     -9.796      0.000      -0.018      -0.012
x6            -1.2951      0.020    -63.416      0.000      -1.335      -1.255
x7             0.3187      0.050      6.335      0.000       0.220       0.417
x8            -1.5241      0.153     -9.968      0.000      -1.824      -1.224
x9            -0.6532      0.089     -7.369      0.000      -0.827      -0.479
x10            2.2027      0.097     22.761      0.000       2.013       2.392
x11           -0.2859      0.175     -1.632      0.103      -0.629       0.057
x12           -1.4271      0.082    -17.501      0.000      -1.587      -1.267
x13            0.7287      0.228      3.196      0.001       0.282       1.176
x14           -1.0476      0.231     -4.536      0.000      -1.500      -0.595
x15           -0.7450      0.099     -7.529      0.000      -0.939      -0.551
x16           -0.2000      0.115     -1.746      0.081      -0.425       0.024
x17            0.1280      0.088      1.454      0.146      -0.045       0.301
x18            1.0949      0.102     10.711      0.000       0.895       1.295
x19           -0.1808      0.096     -1.874      0.061      -0.370       0.008
x20            0.6088      0.092      6.598      0.000       0.428       0.790
x21           -2.0710      0.082    -25.336      0.000      -2.231      -1.911
x22           -0.6801      0.078     -8.673      0.000      -0.834      -0.526
x23           -0.3594      0.077     -4.686      0.000      -0.510      -0.209
x24           -0.8012      0.075    -10.639      0.000      -0.949      -0.654
x25           -2.1963      0.096    -22.780      0.000      -2.385      -2.007
x26           -2.1297      0.170    -12.557      0.000      -2.462      -1.797
x27            1.1287      0.181      6.227      0.000       0.773       1.484
x28            0.1422      0.071      1.990      0.047       0.002       0.282
x29           -1.0961      0.192     -5.711      0.000      -1.472      -0.720
x30           -1.7972      0.076    -23.720      0.000      -1.946      -1.649
x31           -0.0466      0.070     -0.665      0.506      -0.184       0.091
x32           -0.0461      0.096     -0.478      0.633      -0.235       0.143
x33           -0.0677      0.083     -0.812      0.417      -0.231       0.096
x34            0.1496      0.148      1.008      0.313      -0.141       0.440
x35           -0.1154      0.108     -1.071      0.284      -0.327       0.096
x36           -0.0470      0.142     -0.331      0.740      -0.325       0.231
x3

### Model 3 - All variables

In [18]:
# Keep all but time_diff
X3 = np.matrix(bikeshare_machine.drop('time_diff', 1))
y3 = bikeshare_machine['time_diff']

In [38]:
# Split the data into training and testing sets and check the shape
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size = 0.25)
X3_train.shape, X3_test.shape, y3_train.shape, y3_test.shape

((762165, 296), (254056, 296), (762165,), (254056,))

In [39]:
# Fit model
model3 = LinearRegression()
model3.fit(X3_train, y3_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [40]:
pred3 = model3.predict(X3_test)
cvscores_model3 = cross_val_score(model3, X3_train, y3_train, cv=5)
model3_r = model3.score(X3_test, y3_test)
model3_mse = mean_squared_error(y3_test, pred3)
model3_rmse = np.sqrt(model3_mse)
adjustedr3 = 1 - (1-model3_r)*(len(y3_test)-1)/(len(y3_test)-X3_test.shape[1]-1)

In [41]:
cvscores_model3, model3_r, adjustedr3, model3_mse, model3_rmse

(array([  4.62899668e-01,   4.56364879e-01,  -6.31474699e+07,
          4.56765544e-01,   4.55511889e-01]),
 0.45954603608665723,
 0.45891561756625665,
 34.650474212245328,
 5.8864653411232561)

In [23]:
# Use statsmodel to check the statistical side of the model
model3_sm = sm.OLS(y3_train, X3_train.astype(float)).fit()
model3_sm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              time_diff   R-squared:                       0.459
Model:                            OLS   Adj. R-squared:                  0.459
Method:                 Least Squares   F-statistic:                     2255.
Date:                Thu, 14 Dec 2017   Prob (F-statistic):               0.00
Time:                        19:18:13   Log-Likelihood:            -2.4345e+06
No. Observations:              762165   AIC:                         4.870e+06
Df Residuals:                  761877   BIC:                         4.873e+06
Df Model:                         287                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             7.1175      0.010    706.369      0.000       7.098       7.137
x2            -0.0858      0.136     -0.631      0.528      -0.352       0.180
x3             0.0469      0.002     22.998      0.000       0.043       0.051
x4            -0.0038      0.001     -5.270      0.000      -0.005      -0.002
x5            -0.0136      0.001     -9.073      0.000      -0.016      -0.011
x6            -1.2831      0.020    -62.962      0.000      -1.323      -1.243
x7             0.9466      0.043     22.063      0.000       0.863       1.031
x8             0.5789      0.027     21.801      0.000       0.527       0.631
x9            -1.6596      0.153    -10.819      0.000      -1.960      -1.359
x10           -0.7247      0.090     -8.086      0.000      -0.900      -0.549
x11            2.2862      0.097     23.472      0.000       2.095       2.477
x12           -0.1871      0.174     -1.074      0.283      -0.528       0.154
x13           -1.4718      0.082    -17.954      0.000      -1.633      -1.311
x14            0.4709      0.230      2.051      0.040       0.021       0.921
x15           -0.7864      0.226     -3.474      0.001      -1.230      -0.343
x16           -0.7419      0.098     -7.600      0.000      -0.933      -0.551
x17           -0.2351      0.113     -2.086      0.037      -0.456      -0.014
x18            0.0994      0.089      1.118      0.263      -0.075       0.273
x19            1.0533      0.103     10.190      0.000       0.851       1.256
x20           -0.1775      0.097     -1.831      0.067      -0.368       0.013
x21            0.4313      0.093      4.660      0.000       0.250       0.613
x22           -2.1598      0.083    -26.136      0.000      -2.322      -1.998
x23           -0.6745      0.079     -8.580      0.000      -0.829      -0.520
x24           -0.3744      0.076     -4.922      0.000      -0.524      -0.225
x25           -0.8419      0.076    -11.059      0.000      -0.991      -0.693
x26           -2.1380      0.096    -22.181      0.000      -2.327      -1.949
x27           -2.2022      0.170    -12.991      0.000      -2.534      -1.870
x28            1.0956      0.180      6.071      0.000       0.742       1.449
x29            0.1242      0.071      1.748      0.080      -0.015       0.263
x30           -1.3879      0.192     -7.239      0.000      -1.764      -1.012
x31           -1.7804      0.076    -23.344      0.000      -1.930      -1.631
x32           -0.0523      0.070     -0.744      0.457      -0.190       0.085
x33           -0.1435      0.097     -1.479      0.139      -0.334       0.047
x34           -0.1418      0.084     -1.686      0.092      -0.307       0.023
x35            0.0040      0.149      0.027      0.979      -0.289       0.297
x36           -0.1105      0.107     -1.031      0.302      -0.320       0.099
x3

### Model 4 - Ridge with Round 1

In [24]:
# Fit the model with 5 folds
alpha = np.arange(0.0001, 20, 25)
param_grid = {'alpha': alpha}
ridge1 = Ridge(fit_intercept=True)
ridge1_gs = GridSearchCV(ridge1, param_grid, cv=5)
ridge1_gs.fit(X1_train, y1_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([ 0.0001])}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=0)

In [25]:
pred4 = ridge1_gs.predict(X1_test)
ridge1_r = ridge1_gs.score(X1_test, y1_test)
ridge1_mse = mean_squared_error(y1_test, pred4)
ridge1_rmse = np.sqrt(ridge1_mse)
adjustedr4 = 1 - (1-ridge1_r)*(len(y1_test)-1)/(len(y1_test)-X1_test.shape[1]-1)

In [26]:
ridge1_r, ridge1_mse, ridge1_rmse, adjustedr4

(0.45532570566978847,
 35.00140586073227,
 5.9161985988244403,
 0.45473119085948388)

### Ridge with Round 2

In [27]:
# Fit the model with 5 folds
alpha = np.arange(0.0001, 20, 25)
param_grid = {'alpha': alpha}
ridge2 = Ridge(fit_intercept=True)
ridge2_gs = GridSearchCV(ridge2, param_grid, cv=5)
ridge2_gs.fit(X2_train, y2_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([ 0.0001])}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=0)

In [28]:
pred5 = ridge2_gs.predict(X2_test)
ridge2_r = ridge2_gs.score(X2_test, y2_test)
ridge2_mse = mean_squared_error(y2_test, pred5)
ridge2_rmse = np.sqrt(ridge2_mse)
adjustedr5 = 1 - (1-ridge2_r)*(len(y2_test)-1)/(len(y2_test)-X2_test.shape[1]-1)

In [29]:
ridge2_r, ridge2_mse, ridge2_rmse, adjustedr5

(0.45719244700151818,
 34.709163156436745,
 5.8914483072022916,
 0.45656999071172699)

### Ridge with all data

In [30]:
# Fit the model with 5 folds
alpha = np.arange(0.0001, 20, 25)
param_grid = {'alpha': alpha}
ridge3 = Ridge(fit_intercept=True)
ridge3_gs = GridSearchCV(ridge3, param_grid, cv=5)
ridge3_gs.fit(X3_train, y3_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([ 0.0001])}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=0)

In [31]:
pred6 = ridge3_gs.predict(X3_test)
ridge3_r = ridge3_gs.score(X3_test, y3_test)
ridge3_mse = mean_squared_error(y3_test, pred6)
ridge3_rmse = np.sqrt(ridge3_mse)
adjustedr6 = 1 - (1-ridge3_r)*(len(y3_test)-1)/(len(y3_test)-X3_test.shape[1]-1)

In [32]:
ridge3_r, ridge3_mse, ridge3_rmse, adjustedr6

(0.45630905012275935,
 35.04121771007344,
 5.9195622904124798,
 0.45567485578417954)

In [33]:
#bikeshare_machine.to_csv('machine_full.csv')